In [2]:
import os
from dotenv import load_dotenv
# langchain_api="<your-api-key>"
# langchain_project_name="first_project"
# open_ai_key="<your-api-key>"
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('open_ai_key')

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import PyPDFLoader,WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
#creating agents which will combine the tools,llm and prompt
from langchain.agents import create_openai_tools_agent
#to run the agents we need executer
from langchain.agents import AgentExecutor
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
#pulling prompt template from hub
from langchain import hub
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('open_ai_key')
# os.environ['langchain_api']=os.getenv("langchain_api")
# os.environ['LANGCHAIN_TRACING_V2']="true"
from langchain_community.vectorstores import FAISS

In [2]:
#wikipedia tool built by langchain
#creating wikipedia api wrapper and running the wrapper with the uery run basically it is kinda json which will go into the api
#in query run and results will get stored in wiki variable
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
#extracting data from documents
docs=PyPDFLoader(r'D:\azure-session\NCERT-Class-10-History.pdf').load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splitted_pdfs=text_splitter.split_documents(docs)

In [4]:
#extracting data through website
#you can store html from different website as well
web_doc=WebBaseLoader('https://docs.python.org/3/tutorial/classes.html').load()
splitted_html=text_splitter.split_documents(web_doc)

In [5]:
#storing into a db
#we can store it into a persistent storage using chroma db
#db
db_pdf=FAISS.from_documents(splitted_pdfs,OpenAIEmbeddings())
db_web=FAISS.from_documents(splitted_html,OpenAIEmbeddings())
#creating a retriever for pdf and website which will retreive the data from db
retreiver_pdf=db_pdf.as_retriever()
retreiver_web=db_web.as_retriever()
# creating a tool which will contain meta data and the retreiver
#it sees the metadata and only trigger based on metadata
retreival_tool_pdf=create_retriever_tool(retreiver_pdf,'economics',"It is economics chapter in which they have talked about indian economy from 1950-1990 with 5 year plan")
retreival_tool_web=create_retriever_tool(retreiver_web,'Python_documentation','It contains documentation for python')


C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [6]:
#we are defining the series here first it will se the data in website then in pdf then on wkipedi
tools=[retreival_tool_web,retreival_tool_pdf,wiki]

In [7]:
#can use llm from langchain
llm=ChatOpenAI(model='gpt-3.5-turbo',temperature=0.5)

In [8]:
#fetching prompt from hub
#you can customize for your use case
prompt=hub.pull('hwchase17/openai-functions-agent')

In [9]:
#creating an agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [19]:
agent_executer=AgentExecutor(agent=agent,tools=tools,verbose=True,output_parser=output_parser)

In [11]:
import streamlit as st

In [16]:
#creating framework
st.title('Langchain with Open AI API')
input_text=st.text_input('Search the topic you want')

2024-06-10 14:34:33.309 
  command:

    streamlit run C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-06-10 14:34:33.310 Session state does not function when running a script without `streamlit run`


In [15]:
from langchain_core.output_parsers import StrOutputParser

In [16]:
output_parser=StrOutputParser()
chain=

In [20]:
agent_executer.invoke({"input":"Tell me about attention is all you need"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a landmark 2017 research paper authored by eight scientists working at Google, responsible for expanding 2014 attention mechanisms proposed by Bahdanau et al. into a new deep lea"Attention Is All You Need" is a landmark 2017 research paper authored by eight scientists working at Google. It expanded on the attention mechanisms proposed in 2014 by Bahdanau et al. The paper introduced a new deep learning architecture.

> Finished chain.


{'input': 'Tell me about attention is all you need',
 'output': '"Attention Is All You Need" is a landmark 2017 research paper authored by eight scientists working at Google. It expanded on the attention mechanisms proposed in 2014 by Bahdanau et al. The paper introduced a new deep learning architecture.'}

In [20]:
if input_text:
    st.write(chain.invoke({'question':input_text}))

--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\langchain_core\callbacks\manager.py", line 1997, in _configure
    handler = LangChainTracer(
              ^^^^^^^^^^^^^^^^
  File "C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
                            ^^^^^^^^^^^^
  File "C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
              ^^^^^^^^
  File "C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\langsmith\client.py", line 534, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\langsmith\client.py", line 323, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.util



> Entering new AgentExecutor chain...


  File "d:\envs\temp_env\Lib\threading.py", line 1030, in _bootstrap
    self._bootstrap_inner()
  File "d:\envs\temp_env\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "d:\envs\temp_env\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "d:\envs\temp_env\Lib\concurrent\futures\thread.py", line 92, in _worker
    work_item.run()
  File "d:\envs\temp_env\Lib\concurrent\futures\thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
  File "C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\langchain_core\runnables\base.py", line 3140, in transform
    yield from self._transform_stream_with_config(
  File "C:\Users\shiri\AppData\Roaming\Python\Python312\site-packages\langchain_core\runnables\base.py", line 1742, in _transform_stream_with_config
 

KeyError: "Input to ChatPromptTemplate is missing variables {'input'}.  Expected: ['agent_scratchpad', 'input'] Received: ['question', 'intermediate_steps', 'agent_scratchpad']"